# 5G NR CP Autocorrelation Detection Statistic

Computes the normalized cyclic-prefix autocorrelation

$$\hat{\rho} = \frac{|\hat{R}(N_u)|}{\hat{R}(0)}, \quad
\hat{R}(\tau) = \frac{1}{L} \sum_{n=0}^{L-1} x[n]\, x^*[n+\tau]$$

over non-overlapping windows of $L = 2^{17}$ samples from the srsRAN recording.

For 5G NR with 15 kHz SCS at 7.68 MSPS:
- $N_u = 512$ (useful symbol length)
- $N_{\mathrm{cp,normal}} = 36$, $N_{\mathrm{cp,extended}} = 40$ (symbols 0 and 7 per slot)
- Theoretical value: $\rho_{\mathrm{true}} = \Sigma N_{\mathrm{cp}} / L_{\mathrm{slot}} = 512/7680 = 1/15 \approx 0.0667$ (high SNR)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 4)

In [ ]:
# Load SigMF recording (ci32_le: interleaved int32 I/Q)
data_path = 'catkira/1876954_7680KSPS_srsRAN_Project_gnb_short.sigmf-data'
raw = np.fromfile(data_path, dtype='<i4')
x = raw[0::2].astype('float32') + 1j * raw[1::2].astype('float32')
print(f'Loaded {len(x)} complex samples ({len(x)/7.68e6*1e3:.1f} ms at 7.68 MSPS)')

In [ ]:
# Parameters for 5G NR 15 kHz SCS at 7.68 MSPS
fs     = 7.68e6
N_u    = 512          # useful symbol length: fs / 15000
L      = 2**17        # window size: 131072 samples ≈ 17.1 ms

# Theoretical ρ at high SNR:
# 2 extended-CP symbols (40 samples) + 12 normal-CP symbols (36 samples) per slot
# total CP per slot = 2*40 + 12*36 = 512; total samples per slot = 7680
rho_theory = 512 / 7680
print(f'Theoretical ρ (high SNR) = {rho_theory:.4f}  (= 1/15 = {1/15:.4f})')
print(f'Window length L = {L} samples = {L/fs*1e3:.2f} ms')
print(f'Number of non-overlapping windows: {len(x) // L}')

In [ ]:
# Compute ρ̂ for each non-overlapping window
n_windows = len(x) // L
rho = np.empty(n_windows)

for i in range(n_windows):
    seg = x[i*L : (i+1)*L]
    R_lag  = np.mean(seg[:-N_u] * np.conj(seg[N_u:]))   # complex lag-N_u autocorrelation
    R_zero = np.mean(np.abs(seg)**2)                     # power estimate
    rho[i] = np.abs(R_lag) / R_zero

print(f'ρ̂ mean = {rho.mean():.5f}   std = {rho.std():.5f}')
print(f'ρ̂ min  = {rho.min():.5f}   max = {rho.max():.5f}')
print(f'Theoretical ρ = {rho_theory:.5f}')

In [ ]:
# Histogram of the detection statistic
fig, ax = plt.subplots()

ax.hist(rho, bins=12, density=True, alpha=0.7, label=r'$\hat{\rho}$ per window')
ax.axvline(rho_theory, color='C1', linestyle='--', linewidth=1.5,
           label=fr'$\rho_{{\mathrm{{theory}}}} = 1/15 = {rho_theory:.4f}$')
ax.axvline(rho.mean(), color='C2', linestyle='-', linewidth=1.5,
           label=fr'sample mean = {rho.mean():.4f}')

ax.set_xlabel(r'$\hat{\rho} = |\hat{R}(N_u)| / \hat{R}(0)$')
ax.set_ylabel('Density')
ax.set_title(fr'CP autocorrelation statistic, $L = 2^{{17}}$ samples, $N_u = {N_u}$')
ax.legend()
plt.tight_layout()

## Notes

- With $L = 2^{17} = 131072$ samples, each window spans ~17 ms (~1.7 slots).  
  The estimator averages over $L - N_u \approx 130560$ lag pairs, giving a tight distribution.
- The sample mean should be close to $\rho_{\mathrm{theory}} = 1/15 \approx 0.0667$;  
  any small deviation reflects finite-sample bias and SNR < ∞.
- Under H0 (AWGN, no OFDM), $\hat{R}(N_u)$ is the average of $L - N_u$ i.i.d. zero-mean  
  complex Gaussian terms, so $\hat{\rho}$ follows a **Rayleigh** distribution with scale  
  $\sigma_{\mathrm{H0}} \approx 1 / \sqrt{L - N_u} \approx 0.0028$, far below the 5G value.